In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from InsertRead import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

#BME(8/13/2024) Sets criteria for each job to create variables for each dropdown filter option.
#BME(8/13/2024) This maximizes user-friendliness and reduces errors on the user's end.

rescue_criteria = {
    # BME(8/15/2024) Added "All Animals" Option for user to be able to view all animals again, after a filter option
    # has already been selected
    'All Animals': {
        'breeds':[],
        'sex':'',
        'age_range': (0, 999999999)
    },
    'Water': {
        'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'sex' : 'Intact Female',
        'age_range': (26, 156)
    },
    'Mountain or Wilderness': {
        'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'sex' : 'Intact Male',
        'age_range': (26, 156)
    },
    'Disaster or Individual Tracking': {
        'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'sex' : 'Intact Male',
        'age_range': (20, 300)
    }
}

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    
    #BME(8/12/2024) Centered Grazioso Salvare image on Header, Embedded hyperlink in image.
    #BME(8/15/2024) Added code to resize image, it was absurdly large.
    html.Center(html.A(href='https://www.snhu.edu', target="_blank", 
                        children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                                          style={'width': '200px', 'height':'200px'}))),
#     html.Div(id='hidden-div', style={'display':'none'}),
#     html.Center(html.B(html.H1('CS-340 Dashboard'))),
    
    html.Hr(),
    html.Div(),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
#BME(8/12/2024) Added in filter dropdown menu for canine jobs.  Adjusted 8/13/2024
            [html.Label('Filter by Rescue Type'),
            dcc.Dropdown(id='rescue-type-filter',
#BME(8/13/2024) This new options menu for the dropdown uses the dictionary keys as options.
            options=[{'label': key, 'value': key} for key in rescue_criteria.keys()],
#BME(8/12/2024) Default value of none, so that no filter is applied
            value=None,
            placeholder="Select a rescue type from the dropdown.",)
    ]
    ),
   
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         
#BME (8/9/2024) Ensures only a single row is selected
                            row_selectable = "single", 
#BME (8/9/2024) Selects row 0 by default when Dash loads
                            selected_rows = [0],
                            page_size=20,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '1000px', 'height': '500px'},
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H2("Bradley Emerson CS-340 Final Project")
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('rescue-type-filter', 'value'),])
def update_dashboard(selected_rescue_type):
## FIX ME Add code to filter interactive data table with MongoDB queries

        query_filter = {}
    
#BME(8/13/2024) If dropdown menu is used, sets filters to selected criteria

        if selected_rescue_type and selected_rescue_type != 'All Animals':
            criteria = rescue_criteria[selected_rescue_type]
            query_filter = {
                'breed': {'$in': criteria['breeds']},
                'sex_upon_outcome': criteria['sex'],
                'age_upon_outcome_in_weeks': {'$gte': criteria['age_range'][0], '$lte': criteria['age_range'][1]}
            }
        
        filtered_data = db.read(query_filter)
        
#BME(8/13/2024) Removes id column from dataframe
        for record in filtered_data:
            record.pop('_id', None)
#BME(8/13/2024) Debug statement, used to solve 'All Animals' returning 0 records.  Added "and" qualifier 
#BME(8/13/2024) to above "IF" statement to fix the error.
        print(f"Number of records returned: {len(filtered_data)}")
        return pd.DataFrame.from_records(filtered_data).to_dict('records')    
        

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
#BME(8/13/2024) Added new input here to pass filter argument to the update_graphs function
    Input('rescue-type-filter', 'value')])
def update_graphs(viewData, selected_rescue_type):
    
#BME(8/12/2024) Dynamically updates the pie chart using the Data Frame (Filtered)
    dff = pd.DataFrame.from_dict(viewData)
    
#BME(8/13/2024) Creates new variable to determine if filter is applied (Booolean value)
    filter_applied = selected_rescue_type is not None
    
#BME(8/13/2024) Aggregates the pie chart if no filter option is selected, or if the dataframe is empty
    if not filter_applied or dff.empty or selected_rescue_type == 'All Animals':
        aggregated_df = df.groupby('breed').size().reset_index(name='count')
        
#BME(8/13/2024) Groups breeds with <75 dogs into 'Other' so that chart is readable
        threshold = 75
        breed_counts = aggregated_df.set_index('breed')['count']
        few_breeds = breed_counts[breed_counts < threshold]
        breed_counts = breed_counts[breed_counts >= threshold]
        breed_counts['Other'] = few_breeds.sum()
        
        fig = px.pie(names=breed_counts.index, values=breed_counts.values, title='Breed Distribution')
        
        
#BME(8/13/2024) De-aggregates the pie chart if a filter option is selected   
    else:
    
        breed_counts = dff['breed'].value_counts()
  
        fig = px.pie(names=breed_counts.index, values=breed_counts.values, title='Breed Distribution')

#BME(8/13/2024) Resizes the chart within the window for maximum readability
    fig.update_layout(
        autosize=True, 
        width=1000,  
        height=500,  
        margin=dict(l=20, r=20, t=40, b=20),  # Adjust margins to fit the chart
        title_text='Breed Distribution',
        title_x=0.5
    )
    
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    #BME(8/12/2024) Exception handling for null/nil columns
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    #BME (8/11/2024) Added in exception handling to avoid callback errors, returns empty list if viewData is Null/Nil:
    if viewData is None or len(viewData) == 0:
        return []     
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9], style={'font-size': '28px'})
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connected to MongoDB server as aacuser
Dash app running on http://127.0.0.1:13499/
Number of records returned: 9994
Number of records returned: 4
Number of records returned: 17
Number of records returned: 5
Number of records returned: 17
Number of records returned: 5
